In [1]:
print('hi')

hi


In [4]:
pip install tensorflow-gpu

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
pip install keras scikit-learn tqdm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2024-09-08 03:10:38.016681: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-08 03:10:43.863041: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64
2024-09-08 03:10:43.863605: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/

Num GPUs Available:  1


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Замените на номер вашей GPU (если более одной)
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from sklearn.metrics import f1_score


In [5]:
df_TRAIN_RES_1 = pd.read_csv('/home/jupyter/datasphere/project/TRAIN_RES_1.csv')

In [8]:
df = df_TRAIN_RES_1
df = df.iloc[0:200000]

In [7]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Кодирование целевых переменных
le = LabelEncoder()
y_encoded = le.fit_transform(df['job_title'])
print("Прошло кодирование целевых переменных")

# Проверяем количество уникальных значаний
print(f"Количество уникальных значаний в y_encoded: {len(np.unique(y_encoded))}")
print(f"Всего уникальных значаний в y_encoded: {len(set(y_encoded))}")

# Проверяем количество представителей каждого класса
class_counts = np.bincount(y_encoded)

# Создаем новый массив y_filtered, удаляя классы с одним представителем
y_filtered = [x for x in y_encoded if class_counts[x] > 1]
print(f"Прошло удаление классов с одним представителем. Оставшиеся уникальные классы: {len(set(y_filtered))}")

# Создаем новые текстовые данные, учитывая только уникальные классы
text_data = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    job_title = row['job_title']
    class_id = le.transform([job_title])[0]
    if class_id in y_filtered:
        combined_text = f"{row['achievements_modified']} {row['demands']}"
        text_data.append(combined_text)
print("Прошло создание текстовых данных")

# Разделяем данные
train_text, val_text, train_labels, val_labels = train_test_split(
    text_data,
    y_filtered,
    random_state=42,
    test_size=0.2,
    stratify=y_filtered
)
print("Прошло разделение данных")

# Преобразуем train_labels и val_labels в NumPy массивы
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

# Создаем токенизатор
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_text)

# Преобразуем тексты в последовательности слов
train_sequences = tokenizer.texts_to_sequences(train_text)
val_sequences = tokenizer.texts_to_sequences(val_text)

# Добавляем нулевые значения для коротких последовательностей
pad_length = 200
padded_train = pad_sequences(train_sequences, maxlen=pad_length)
padded_val = pad_sequences(val_sequences, maxlen=pad_length)

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Создаем модель
model = Sequential([
    Embedding(input_dim=5000, output_dim=128),
    LSTM(units=64, dropout=0.2),
    Dense(64, activation='relu'),
    Dense(len(set(y_filtered)), activation='softmax')
])

# Компилируем модель
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Функция для преобразования меток
def transform_labels(labels):
    return np.array([min(max(label, 0), len(label_mapping) - 1) for label in labels])

# Создаем словарь для замены меток
label_mapping = {k: v for k, v in zip(range(len(set(y_filtered))), set(y_filtered))}
print(f"Создано словарь label_mapping с {len(label_mapping)} элементами.")

# Обучаем модель
print("Запуск обучения модели...")
with tqdm(total=train_labels.shape[0], desc="Обучение модели") as pbar:
    for epoch in range(10):
        transformed_train_labels = transform_labels(train_labels)
        transformed_val_labels = transform_labels(val_labels)
        history = model.fit(padded_train, transformed_train_labels, epochs=1, 
                            validation_data=(padded_val, transformed_val_labels), 
                            verbose=0)
        val_loss, val_accuracy = model.evaluate(padded_val, transformed_val_labels, verbose=0)
        pbar.set_postfix({"Val Loss": f"{val_loss:.4f}", "Val Acc": f"{val_accuracy:.4f}"})
        pbar.update(train_labels.shape[0])
    
    pbar.close()

print("Прошло обучение модели")

# Генерация предсказаний
predictions = model.predict(padded_val)
predicted_classes = predictions.argmax(axis=-1)

print("Прошло генерация предсказаний")

# Расчет F1-score
f1_micro = f1_score(transformed_val_labels, predicted_classes, average='micro')

print(f"F1-score (micro): {f1_micro:.3f}")

model_path = '/home/jupyter/datasphere/project/model_new.h5'
model.save(model_path)
print(f"Модель успешно сохранена в файл {model_path}")

Прошло кодирование целевых переменных
Количество уникальных значаний в y_encoded: 638
Всего уникальных значаний в y_encoded: 638
Прошло удаление классов с одним представителем. Оставшиеся уникальные классы: 483


100%|██████████| 50000/50000 [00:29<00:00, 1719.36it/s]


Прошло создание текстовых данных
Прошло разделение данных
Создано словарь label_mapping с 483 элементами.
Запуск обучения модели...


Обучение модели: 398760it [04:14, 1564.89it/s, Val Loss=1.9231, Val Acc=0.5843]                         


Прошло обучение модели
312/312 [==============================] - 2s 4ms/step
Прошло генерация предсказаний
F1-score (micro): 0.584
Модель успешно сохранена в файл /home/jupyter/datasphere/project/model.h5


In [16]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Путь к загружаемой модели
model_path = '/home/jupyter/datasphere/project/model.h5'

# Загрузка модели
model = load_model(model_path)
print(f"Модель успешно загружена из файла {model_path}")

# Загрузка нового файла с данными (например, в формате CSV)
new_data_path = '/home/jupyter/datasphere/project/TEST_RES.csv'  # Укажите путь к вашему новому файлу
new_df = pd.read_csv(new_data_path)

# Предполагается, что в новом файле есть столбцы 'achievements_modified' и 'demands'
# Создание текстовых данных для предсказания
new_text_data = []
for _, row in new_df.iterrows():
    combined_text = f"{row['achievements_modified']} {row['demands']}"
    new_text_data.append(combined_text)

# Токенизация новых текстов
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(new_text_data)  # Обучить токенизатор на новых данных

# Преобразование текстов в последовательности
new_sequences = tokenizer.texts_to_sequences(new_text_data)

# Дополнение последовательностей до одинаковой длины
pad_length = 200  # Должно совпадать с длиной, использованной при обучении
padded_new = pad_sequences(new_sequences, maxlen=pad_length)

# Применение модели к новым данным
predictions = model.predict(padded_new)
predicted_classes = np.argmax(predictions, axis=-1)

# Если хотите преобразовать предсказанные классы обратно в названия
predicted_job_titles = le.inverse_transform(predicted_classes)

# Добавление предсказаний в DataFrame для вывода
new_df['job_title'] = predicted_job_titles

# Сохранение результатов в новый файл
output_path = '/home/jupyter/datasphere/project/output_file2.csv'  # Укажите путь к выходному файлу
new_df.to_csv(output_path, index=False)
print(f"Предсказания успешно сохранены в файл {output_path}")


Модель успешно загружена из файла /home/jupyter/datasphere/project/model.h5
62/62 [==============================] - 1s 4ms/step
Предсказания успешно сохранены в файл /home/jupyter/datasphere/project/output_file2.csv
